In [0]:
import pandas as pd

df = pd.read_csv('tweets_translated_chn.csv')
df.shape

(61831, 8)

In [0]:
df.head()

,Unnamed: 0,Unnamed: 0.1,id_str,screen_name,raw_text,chn_text,eng_text,translated_chn
0,0,0,1193152895337689088,YXSzzzz,@gww067 Haha there is no way Ryza is not cute 😂,NaN,Haha there is no way Ryza is not cute,哈哈，不可能瑞紮不可愛
1,1,1,1193152895337689088,YXSzzzz,乳不平何以平天下!\nLess busty testing. https://t.co/Iy...,乳不平何以平天下!,Less busty testing.,不太繁忙的測試。
2,2,2,1193152895337689088,YXSzzzz,@HrJasn 乳不巨何以聚人心 (誤,乳不巨何以聚人心誤,NaN,NaN
3,3,3,1193152895337689088,YXSzzzz,因為蠻多人問。。。我就統一發個 😂\nBecuz too many people ask ....,因為蠻多人問。。。我就統一發個這個是我自己無聊畫的，不是…,Becuz too many people ask ... I will post thi...,因為太多人問...我會張貼這個Mod這是我的圖紙，而fe
4,4,4,1193152895337689088,YXSzzzz,PS: it was not my update. but good to see more...,NaN,PS: it was not my update. but good to see more...,PS：這不是我的更新。但很高興看到更多的人做模組


In [0]:
df.screen_name.unique()

array(['YXSzzzz', 'GreenBright2', 'ZhuLiuHai1', 'SangPu15', 'BlessHK3',
       '0318May', '2nmnmcvkhKwByww', 'RainbowRainRai1', 'kacavawu',
       'Lucien86734903', 'yueming02341069', 'susanzh77455188',
       'Abby23079586', 'CherieHuang3', 'yuenck3', 'hkunderwearmen',
       'Iming9999', 'lee_tungtung', 'ZazousLes', 'Heerfei',
       'AnnaRao32559926', 'amyfox2009', 'pyequality', 'dgchuanghe',
       'EmmaHaoTheBest', 'kevin77757997', 'YingShi55', 'Wangcat11',
       'KP_Taipei', 'Nan_Is_Me', 'kelvinchan4335', 'EdrWx',
       'Shevonne1207', 'steffenroski', 'cla12a2013', 'bluerightking',
       'thenapoleonhung', 'easoncxz', 'siumou', 'yipcw',
       'SVWomanEngineer', 'kk23wong', 'calvynation', 'AndyCoach',
       'denistsao', 'Leifson', 'hongliji', 'sunbear', 'TrueOfLifeTP',
       'poiesis'], dtype=object)

In [0]:
len(df.screen_name.unique())

50

## Vectorize the tweets

In [0]:
!pip3 install polyglot PyICU pycld2 morfessor

     |████████████████████████████████| 133kB 3.5MB/s 
     |████████████████████████████████| 225kB 47.2MB/s 
     |████████████████████████████████| 41.4MB 56kB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52560 sha256=3eeb89d8ed2bb32d1782fb3b2fe751926afe6c8defd0dad393dba3fe445fdf5b
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b83a305f39ab047986d9da
  Created wheel for PyICU: filename=PyICU-2.4.2-cp36-cp36m-linux_x86_64.whl size=1245270 sha256=efcf5c48c0751db26f4f28887e3f0f6045d027f222aaae70dbf24de3212a9a2b
  Stored in directory: /root/.cache/pip/wheels/d6/78/89/08a15173ae1905de95ce9a5f55b17259ca5b462ed11a4c068c
  Created wheel for pycld2: filename=pycld2-0.41-cp36-cp36m-linux_x86_64.whl size=9833412 sha256=faf669a3ffc2d39120c09f9017dbbffe0c9b3566d92c26e22aad06388276410e
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built polyglot PyI

### Prepare Monolingual Texts

In [0]:
df['monolingual_text'] = df[['chn_text', 'eng_text']].values.tolist()
df = df.explode('monolingual_text')
df = df.reset_index(drop=True)
df = df.dropna(subset=['monolingual_text'])
df = df.reset_index(drop=True)
df.head()

,Unnamed: 0,id_str,screen_name,raw_text,chn_text,eng_text,monolingual_text
0,0,1193152895337689088,YXSzzzz,@gww067 Haha there is no way Ryza is not cute 😂,NaN,Haha there is no way Ryza is not cute,Haha there is no way Ryza is not cute
1,1,1193152895337689088,YXSzzzz,乳不平何以平天下!\nLess busty testing. https://t.co/Iy...,乳不平何以平天下!,Less busty testing.,乳不平何以平天下!
2,1,1193152895337689088,YXSzzzz,乳不平何以平天下!\nLess busty testing. https://t.co/Iy...,乳不平何以平天下!,Less busty testing.,Less busty testing.
3,2,1193152895337689088,YXSzzzz,@HrJasn 乳不巨何以聚人心 (誤,乳不巨何以聚人心誤,NaN,乳不巨何以聚人心誤
4,3,1193152895337689088,YXSzzzz,因為蠻多人問。。。我就統一發個 😂\nBecuz too many people ask ....,因為蠻多人問。。。我就統一發個這個是我自己無聊畫的，不是…,Becuz too many people ask ... I will post thi...,因為蠻多人問。。。我就統一發個這個是我自己無聊畫的，不是…


In [0]:
df.shape

(73860, 7)

### Prepare Translated Monolingual Texts

In [0]:
df['chn_text'].fillna(df['translated_chn'], inplace=True)
df.chn_text = df.chn_text.astype(str)
df.head()

,Unnamed: 0,Unnamed: 0.1,id_str,screen_name,raw_text,chn_text,eng_text,translated_chn
0,0,0,1193152895337689088,YXSzzzz,@gww067 Haha there is no way Ryza is not cute 😂,哈哈，不可能瑞紮不可愛,Haha there is no way Ryza is not cute,哈哈，不可能瑞紮不可愛
1,1,1,1193152895337689088,YXSzzzz,乳不平何以平天下!\nLess busty testing. https://t.co/Iy...,乳不平何以平天下!,Less busty testing.,不太繁忙的測試。
2,2,2,1193152895337689088,YXSzzzz,@HrJasn 乳不巨何以聚人心 (誤,乳不巨何以聚人心誤,NaN,NaN
3,3,3,1193152895337689088,YXSzzzz,因為蠻多人問。。。我就統一發個 😂\nBecuz too many people ask ....,因為蠻多人問。。。我就統一發個這個是我自己無聊畫的，不是…,Becuz too many people ask ... I will post thi...,因為太多人問...我會張貼這個Mod這是我的圖紙，而fe
4,4,4,1193152895337689088,YXSzzzz,PS: it was not my update. but good to see more...,PS：這不是我的更新。但很高興看到更多的人做模組,PS: it was not my update. but good to see more...,PS：這不是我的更新。但很高興看到更多的人做模組


### Prepare X, y, CV

In [0]:
from polyglot.text import Text
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import KFold

X = df.chn_text
y = df.screen_name

def ploy_tokenizer(raw_text):
    return Text(raw_text).words

count_vectorizer = CountVectorizer(tokenizer=ploy_tokenizer)
tf_transformer = TfidfTransformer()

kf = KFold(n_splits=10, shuffle=True, random_state=1)

## Classification

In [0]:
import logging

from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics

logging.getLogger().setLevel(logging.ERROR)

nb = MultinomialNB()
lr = LogisticRegression(solver='liblinear', multi_class='auto', class_weight='balanced', verbose=1)
svm = LinearSVC(class_weight='balanced', verbose=1)

scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

def tokenize_test_pipeline(model, dataset_X=X, dataset_y=y):
    pipe = Pipeline([
        ('count', count_vectorizer),
        ('tfidf', tf_transformer),
        ('clf', globals()[model]),
    ])

    params = {
        'count__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'count__analyzer': ['word', 'char'],
    }

    algorithm = ''
    if model.startswith('lr'):
        algorithm = 'Logistic Regression'
    elif model.startswith('nb'):
        algorithm = 'Multinomial Naive Bayes'
    elif model.startswith('svm'):
        algorithm = 'Linear SVM'

    gd_clf = GridSearchCV(pipe, params, cv=10, scoring=scoring, n_jobs=-1, refit='accuracy', verbose=1)
    gd_clf.fit(dataset_X, dataset_y)

    print(algorithm)
    print('------------------')
    print('Scores: ')
    print(gd_clf.cv_results_)
    print('Best params:')
    print(gd_clf.best_params_)
    print('Best score:')
    print(gd_clf.best_score_)

### Logistic Regression

In [0]:
tokenize_test_pipeline(model='lr')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 56.2min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 89.4min finished


[LibLinear]Logistic Regression
------------------
Scores: 
{'mean_fit_time': array([ 60.14813883, 149.37297668, 272.25913916,  46.75755084,
       171.21401787, 323.37935736]), 'std_fit_time': array([ 1.36537253,  2.84256665, 11.97859924,  2.11104568,  2.63658098,
        6.7273698 ]), 'mean_score_time': array([ 6.82493012, 12.24646518, 17.77140684,  1.24236243,  3.23189859,
        5.13974364]), 'std_score_time': array([0.19266   , 0.32581927, 0.83195158, 0.02796554, 0.12642395,
       0.66589351]), 'param_count__analyzer': masked_array(data=['word', 'word', 'word', 'char', 'char', 'char'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_count__ngram_range': masked_array(data=[(1, 1), (1, 2), (1, 3), (1, 1), (1, 2), (1, 3)],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'count__analyzer': 'word', 'count__ngram_range': (1, 1)}, {'count__an

### SVM

In [0]:
tokenize_test_pipeline(model='svm')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 28.2min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 38.6min finished


[LibLinear]Linear SVM
------------------
Scores: 
{'mean_fit_time': array([ 27.82304461,  65.15149863, 114.56113191,  49.33997896,
        64.51706111,  91.97935462]), 'std_fit_time': array([0.36800456, 1.26559449, 2.83438217, 1.41091034, 1.17297237,
       2.15716559]), 'mean_score_time': array([ 6.67590892, 12.16759541, 17.59144361,  1.25358963,  3.24936681,
        5.05684392]), 'std_score_time': array([0.13684759, 0.39998259, 0.63652009, 0.0387192 , 0.09342281,
       0.74381784]), 'param_count__analyzer': masked_array(data=['word', 'word', 'word', 'char', 'char', 'char'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_count__ngram_range': masked_array(data=[(1, 1), (1, 2), (1, 3), (1, 1), (1, 2), (1, 3)],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'count__analyzer': 'word', 'count__ngram_range': (1, 1)}, {'count__analyzer': 'word'

### Translated ENG Logistic Regression

In [0]:
tokenize_test_pipeline(model='lr')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 26.1min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 36.7min finished


[LibLinear]Logistic Regression
------------------
Scores: 
{'mean_fit_time': array([ 25.6846704 ,  74.34284422, 146.71994419,  11.7750067 ,
        50.77382355, 101.68126271]), 'std_fit_time': array([1.08159125, 1.02032594, 1.83717831, 0.56247878, 0.73258752,
       0.54668601]), 'mean_score_time': array([ 3.20612438,  6.6951107 , 10.44163733,  0.79150991,  2.35312645,
        4.06918824]), 'std_score_time': array([0.1424879 , 0.27940092, 0.38629754, 0.04111449, 0.11567768,
       0.70537673]), 'param_count__analyzer': masked_array(data=['word', 'word', 'word', 'char', 'char', 'char'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_count__ngram_range': masked_array(data=[(1, 1), (1, 2), (1, 3), (1, 1), (1, 2), (1, 3)],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'count__analyzer': 'word', 'count__ngram_range': (1, 1)}, {'count__analyzer

### Translated ENG SVM

In [0]:
tokenize_test_pipeline(model='svm')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 17.4min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 22.6min finished


[LibLinear]Linear SVM
------------------
Scores: 
{'mean_fit_time': array([14.2969492 , 36.01741168, 71.13299069, 40.42135382, 38.09852674,
       41.97550185]), 'std_fit_time': array([0.24180769, 0.48819387, 1.22297034, 3.52377568, 0.42097591,
       0.47395641]), 'mean_score_time': array([ 3.27321918,  6.72942572, 10.43951981,  0.76281691,  2.32593699,
        4.06143386]), 'std_score_time': array([0.12701345, 0.30489256, 0.46002468, 0.03265929, 0.11217818,
       0.53436629]), 'param_count__analyzer': masked_array(data=['word', 'word', 'word', 'char', 'char', 'char'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_count__ngram_range': masked_array(data=[(1, 1), (1, 2), (1, 3), (1, 1), (1, 2), (1, 3)],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'count__analyzer': 'word', 'count__ngram_range': (1, 1)}, {'count__analyzer': 'word', 'cou

### Translated CHN Logistic Regression

In [0]:
tokenize_test_pipeline(model='lr')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 67.1min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 106.8min finished


[LibLinear]Logistic Regression
------------------
Scores: 
{'mean_fit_time': array([ 69.37375982, 188.26056206, 331.17762198,  57.87744391,
       189.37398825, 394.73208094]), 'std_fit_time': array([ 5.14807234,  4.78311496, 11.26311802,  4.72187133,  8.56752815,
        8.42270446]), 'mean_score_time': array([ 7.63888748, 13.58429797, 19.13383217,  1.07727373,  2.64178829,
        4.32317219]), 'std_score_time': array([0.30444688, 0.61741762, 0.62095566, 0.05249818, 0.21240878,
       0.29613198]), 'param_count__analyzer': masked_array(data=['word', 'word', 'word', 'char', 'char', 'char'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_count__ngram_range': masked_array(data=[(1, 1), (1, 2), (1, 3), (1, 1), (1, 2), (1, 3)],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'count__analyzer': 'word', 'count__ngram_range': (1, 1)}, {'count__an

### Translated CHN SVM

In [0]:
tokenize_test_pipeline(model='svm')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 40.0min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 59.3min finished


[LibLinear]Linear SVM
------------------
Scores: 
{'mean_fit_time': array([ 36.74613323, 114.18015513, 211.60840144,  26.01608417,
        76.03504174, 193.95257936]), 'std_fit_time': array([ 1.06753452,  4.28348639, 10.43651527,  1.69808619,  3.9286417 ,
        5.92905862]), 'mean_score_time': array([ 7.76997623, 13.46118484, 19.32058494,  1.13208673,  2.74970307,
        4.30341444]), 'std_score_time': array([0.29386989, 0.47636575, 0.64667953, 0.05233927, 0.16295863,
       0.64612305]), 'param_count__analyzer': masked_array(data=['word', 'word', 'word', 'char', 'char', 'char'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_count__ngram_range': masked_array(data=[(1, 1), (1, 2), (1, 3), (1, 1), (1, 2), (1, 3)],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'count__analyzer': 'word', 'count__ngram_range': (1, 1)}, {'count__analyzer': 